# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [5]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [17]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))


# Incorporate resource constraints for each good j
# Define total endowments
endowments = torch.tensor([30.0, 30.0, 30.0], dtype = torch.float32)

In [18]:
# Social planner's objective, with RC enforced
def social_planner_objective(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j, omega_j)
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative


### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [31]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1500
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if iteration % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

print("Optimal allocations:")
print(final_allocations)
# print("Total allocations per good:")
# print(final_allocations.sum(axis=0))
# print("Endowments:")
# print(endowments.numpy())


Iteration 0: Objective = 16.002307891845703
Iteration 100: Objective = 40.352108001708984
Iteration 200: Objective = 88.16224670410156
Iteration 300: Objective = 156.52291870117188
Iteration 400: Objective = 238.8422393798828
Iteration 500: Objective = 331.4263000488281
Iteration 600: Objective = 432.4744873046875
Iteration 700: Objective = 540.9359130859375
Iteration 800: Objective = 569.283935546875
Iteration 900: Objective = 569.2868041992188
Iteration 1000: Objective = 569.2880249023438
Iteration 1100: Objective = 569.28955078125
Iteration 1200: Objective = 569.2914428710938
Iteration 1300: Objective = 569.2937622070312
Iteration 1400: Objective = 569.2967529296875
Optimal allocations:
[[10.137773   9.8984585 10.027572 ]
 [ 9.845664  10.18302    9.949477 ]
 [10.021302   9.9232645 10.027695 ]]


In [25]:
type(final_allocations)

numpy.ndarray

### Case II: Heterogeneous Agents 
> $m = n = 3$

Parameters $\alpha_j, \omega_j^i$
Social weights $\lambda_j$

In [ ]:
# =============== DEFINE PARAMETERS ===============
alpha_j_het = torch.tensor([1.0, 0.8, 1.2], dtype = torch.float32) 
omega_j_het = torch.tensor([[0.3, 0.5, 0.7]] * n, dtype = torch.float32)
lambda_i_het = torch.tensor([0.9, 1.0, 1.01], dtype = torch.float32)  # Pareto weights

